In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm
import json

In [4]:
with open('deep_fried_chicken_data.json', 'w') as f:
    json.dump({}, f)

In [ ]:
#27,697 restaurants, as of March 5


In [32]:
data={}
for j in tqdm(range(0,9)):
    url = "https://www.openrice.com/en/hongkong/restaurants/dish/korean-fried-chicken?page="
    page = str(j+1)
    headers = requests.utils.default_headers()
    headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
    response = requests.get(url+page, headers=headers)
    response = response.text
    soup = BeautifulSoup(response, "html.parser")
    
    content = soup.find(class_='sr1-listing-content-cells pois-restaurant-list js-poi-list-content-cell-container')
    
    data[j] = content.prettify()
    
    if j % 8 == 0:
        with open('deep_fried_chicken_data.json') as f:
            file = json.load(f)

        file.update(data)

        with open('deep_fried_chicken_data.json', 'w') as f:
            json.dump(file, f)
            
#         data = {}

  0%|          | 0/9 [00:00<?, ?it/s]

In [3]:
# with open('data.json') as f:
#     data = json.load(f)

In [33]:
from tqdm import tqdm
from tqdm.notebook import trange, tqdm

In [26]:
# !pip install tqdm --upgrade

     |████████████████████████████████| 75 kB 290 kB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.31.1
    Uninstalling tqdm-4.31.1:
      Successfully uninstalled tqdm-4.31.1
You should consider upgrading via the '/Users/Kevin/anaconda3/bin/python -m pip install --upgrade pip' command.


In [34]:
df = pd.DataFrame(columns =['name','review','bookmark','smile','sad','address','district','cuisine','dish'])

In [38]:
for k in range(len(data)):
    soup = BeautifulSoup(data[k], "html.parser")
    for i in range(len(soup.find_all(class_='title-name'))):
        rest = {}
        rest['name'] = soup.find_all(class_='title-name')[i].text.strip()
        try:
            review = soup.find_all(class_="counters-container")[i].text.strip()
            rest['review'] = int(*re.findall('\d+', review))
        except:
            rest['review'] = 'N/A'
        rest['bookmark'] = soup.find_all(class_="text bookmarkedUserCount js-bookmark-count")[i]['data-count']
        try:
            rest['smile'] = soup.find_all(class_="emoticon-container smile-face pois-restaurant-list-cell-content-right-info-rating-happy")[i].text.strip()
        except:
            rest['smile'] = 'N/A'
        try:
            rest['sad'] = soup.find_all(class_="emoticon-container sad-face pois-restaurant-list-cell-content-right-info-rating-sad")[i].text.strip()
        except:
            rest['sad'] = 'N/A' 
        
        rest['address'] = soup.find_all(class_='icon-info-wrapper')[i].find('span').find(text=True).strip()
        rest['district'] = soup.find_all(class_='icon-info-wrapper')[i].find('a').text
        rest['cuisine'] = soup.find_all(class_="icon-info icon-info-food-name")[i].find_all('a')[0].text
        rest['dish'] = soup.find_all(class_="icon-info icon-info-food-name")[i].find_all('a')[1].text
        df = df.append(rest, ignore_index=True)

In [39]:
df.to_csv('deep_dried_chicken_restaurant.csv')

In [47]:
df = pd.read_csv('deep_dried_chicken_restaurant.csv', index_col=0)

In [115]:
with open('deep_dried_chicken_restaurant_latlong.json', 'w') as f:
    json.dump({}, f)

In [42]:
re_dict={}

In [114]:
def ogcioparser_dict(address):
    data = {"q":address, "n":2} #n means only send 1 
    headers ={"Accept": "application/json"}
    api_url = "https://www.als.ogcio.gov.hk/lookup"
    res = requests.post(api_url, data=data, headers=headers, timeout=(5, 14))
    add = json.loads(res.text)
    add_dict = {}
    add_dict['Latitude'] = add['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation']['Latitude']    
    add_dict['Longitude'] = add['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation']['Longitude']
    add_dict['Easting'] = add['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation']['Easting']
    add_dict['Northing'] = add['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation']['Northing']
    return add_dict

In [101]:
addresses = df['address']

In [102]:
sleep_counter = 0

In [117]:
all_dict = {}
working_dict = {}
for i, a in enumerate(tqdm(addresses)):
    parse = ogcioparser_dict(a)
    all_dict[i] = parse
    working_dict[i] = parse
#     sleep_counter +=1
    if i == len(addresses)-1:
#         sleep(randint(5,10))

        with open('deep_dried_chicken_restaurant_latlong.json') as f:
            file = json.load(f)

        file.update(working_dict)

        with open('deep_dried_chicken_restaurant_latlong.json', 'w') as f:
            json.dump(file, f)
            
        working_dict = {}

100%|██████████| 418/418 [12:36<00:00,  1.75s/it]


In [118]:
df_parse = pd.DataFrame.from_dict(all_dict, orient='index')

In [119]:
df_parse.to_csv('parse_fried.csv')

In [120]:
df_parse

,Latitude,Longitude,Easting,Northing
0,22.31578,114.17057,835619,819643
1,22.2783,114.18316,836916,815493
2,22.29958,114.17455,836028,817849
3,22.2777,114.1822,836817,815426
4,22.28077,114.18278,836877,815766
5,22.29795,114.17465,836039,817669
6,22.27911,114.18164,836759,815583
7,22.31781,114.16991,835551,819868
8,22.28276,114.15394,833905,815987
9,22.27969,114.18385,836987,815647


In [121]:
result = pd.concat([df, df_parse], axis=1)

In [128]:
result.head()

,name,review,bookmark,smile,sad,address,district,cuisine,dish,Latitude,Longitude,Easting,Northing
0,Yadllie Plate,1235.0,80549,944.0,52.0,"11/F, CTMA Centre, 1 Sai Yeung Choi Street,",\n Mong Kok\n,\n Korean Fried Chicken\n,\n Korean\n,22.31578,114.17057,835619,819643
1,Dodam Chicken (利舞臺廣場),1080.0,41429,912.0,22.0,"18/F., Shop A, Lee Theatre Plaza, 99 Percival ...",\n Causeway Bay\n,\n Korean Fried Chicken\n,\n Korean\n,22.2783,114.18316,836916,815493
2,Outdark (厚福街),658.0,36973,544.0,14.0,"17/F, H8, 8 Hau Fook Street,",\n Tsim Sha Tsui\n,\n Korean Fried Chicken\n,\n Korean\n,22.29958,114.17455,836028,817849
3,Sodam Chicken,1330.0,57055,1018.0,66.0,"7/F, 3 Yiu Wa Street, Bartlock Centre,",\n Causeway Bay\n,\n Korean Fried Chicken\n,\n Korean\n,22.2777,114.1822,836817,815426
4,Bingo & Cook,1148.0,34010,991.0,22.0,"2/F, L'Hart, 487-489 Lockhart Road,",\n Causeway Bay\n,\n Korean Fried Chicken\n,\n Korean\n,22.28077,114.18278,836877,815766


In [129]:
result['district'] = result['district'].apply(lambda x: x.strip())
result['cuisine'] = result['cuisine'].apply(lambda x: x.strip())
result['dish'] = result['dish'].apply(lambda x: x.strip())

In [ ]:
result.to_csv()